LibAUC Experiments

In [1]:
from libauc.models import resnet18
from libauc.losses import AUCMLoss

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as torch_data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator


from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import densenet121 as DenseNet121
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore") 

In [3]:
data_flag = 'breastmnist'
# data_flag = 'chestmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [4]:
task

'binary-class'

In [7]:
data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5),
        ])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
val_dataset = DataClass(split='val', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = torch_data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch_data.DataLoader(dataset=val_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = torch_data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz
Using downloaded and verified file: C:\Users\Hari\.medmnist\breastmnist.npz


In [13]:
lr = 0.01 # using smaller learning rate is better
epoch_decay = 2e-5
weight_decay = 1e-7
margin = 1.0

model = resnet18(num_classes=2)
model = model.cuda()
# criterion = nn.CrossEntropyLoss()  
# optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=criterion, 
                 lr=lr, 
                 margin=margin, 
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)
CE_loss = nn.CrossEntropyLoss()
# training
best_val_auc = 0 
for epoch in range(1001):
    train_losses = []
    for idx, data in enumerate(train_loader):
      print("Iteration started")
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = criterion(y_pred, train_labels.type(torch.LongTensor).cuda())
      train_losses.append(loss.item()/len(data))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      break

    print("Epoch : {:03d}  Train Loss : {:.5f} ".format(epoch, np.mean(train_losses)), end='')
    model.eval()
    with torch.no_grad():    
        test_pred = []
        test_true = [] 
        test_losses = []
        test_CE_losses = []
        for jdx, data in enumerate(val_loader):
            test_data, test_labels = data
            test_data = test_data.cuda()
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_labels.numpy())
            test_losses.append(criterion(y_pred, test_labels.squeeze().type(torch.LongTensor).cuda()).item() / len(data))
            # test_CE_losses.append(CE_loss(y_pred, test_labels.squeeze().float().cuda()).cpu())

        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)

        # val_auc_mean = 0
        # for i in range(14):
        val_auc =  roc_auc_score(test_true, test_pred[:,1]) 
        # val_auc_mean/=14
        print("Val Loss : {:.5f}   ".format(np.mean(test_losses)), end = '')
        model.train()

        if best_val_auc < val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), 'pretrained_model.pth')

        print ('BatchID= {}   Val_AUC={:.4f}   Best_Val_AUC={:.4f}'.format(
            idx, val_auc, best_val_auc ))
          
    print("Epoch : {}".format(epoch))

Iteration started
Epoch : 000  Train Loss : 0.07435 Val Loss : 0.00391   BatchID= 0   Val_AUC=0.5589   Best_Val_AUC=0.5589
Epoch : 0
Iteration started
Epoch : 001  Train Loss : 0.07070 Val Loss : 0.00783   BatchID= 0   Val_AUC=0.6349   Best_Val_AUC=0.6349
Epoch : 1
Iteration started
Epoch : 002  Train Loss : 0.06626 Val Loss : 0.01184   BatchID= 0   Val_AUC=0.5196   Best_Val_AUC=0.6349
Epoch : 2
Iteration started
Epoch : 003  Train Loss : 0.06046 Val Loss : 0.01608   BatchID= 0   Val_AUC=0.4912   Best_Val_AUC=0.6349
Epoch : 3
Iteration started
Epoch : 004  Train Loss : 0.05055 Val Loss : 0.02063   BatchID= 0   Val_AUC=0.4929   Best_Val_AUC=0.6349
Epoch : 4
Iteration started
Epoch : 005  Train Loss : 0.04325 Val Loss : 0.02556   BatchID= 0   Val_AUC=0.4887   Best_Val_AUC=0.6349
Epoch : 5
Iteration started
Epoch : 006  Train Loss : 0.05455 Val Loss : 0.03132   BatchID= 0   Val_AUC=0.4904   Best_Val_AUC=0.6349
Epoch : 6
Iteration started
Epoch : 007  Train Loss : 0.03965 Val Loss : 0.0368

: 

In [10]:
test_true.shape

(78, 1)

In [12]:
test_pred.shape

(78, 2)